IMPORTING LIBRARIES

In [139]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.models import load_model
import librosa
from IPython.display import Audio
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

LOADING MODEL

In [140]:
model=load_model('E:/final year project/models/model_0.52.keras')

LOADING THE AUDIO FILE

In [141]:



file_path = r"E:\final year project\dataset\Angry\1022_DFA_ANG_XX.wav"
Audio(file_path)
audio, sample_rate = librosa.load(file_path, sr=None)  

print(f"Audio Shape: {audio.shape}, Sample Rate: {sample_rate}")


Audio Shape: (46980,), Sample Rate: 16000


In [142]:
def get_melspec(y,sr,n_fft,n_mels,hop_length,fmin,maxlen):
    mel_spec=librosa.feature.melspectrogram(y=y,sr=sr,n_fft=n_fft,n_mels=n_mels,hop_length=hop_length,fmin=fmin)
    mel_spec_db=librosa.power_to_db(mel_spec,ref=np.max)
    return pad_sequences(mel_spec_db,maxlen=maxlen,truncating="post",padding="post")

In [143]:
FRAME_LENGTH=2048
HOP_LENGTH=512
N_MFCC=13
N_FFT=1024
MAX_LENGTH=130
N_MELS=128
FMIN=50
melspecs=get_melspec(audio,sample_rate,N_FFT,N_MELS,HOP_LENGTH,FMIN,MAX_LENGTH)
melspecs=np.array(melspecs)
melspecs.shape

(128, 130)

In [144]:
def normalise_features(feature):
    mean=np.mean(feature,axis=0)
    std=np.std(feature,axis=0)
    return (feature-mean)/(std+1e-9)

In [145]:
normalisied_melspecs=np.array(normalise_features(melspecs))
normalisied_melspecs.shape

(128, 130)

In [146]:
print(model.input_shape)
normalisied_melspecs=normalisied_melspecs.reshape(1,128,130,1)
pred=model.predict(normalisied_melspecs)
pred

(None, 128, 130, 1)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 270ms/step


array([[0.00360219, 0.13760597, 0.208976  , 0.01619476, 0.02941798,
        0.60420316]], dtype=float32)

In [147]:
label=np.argmax(pred,axis=1)[0]
print(label)
classes=['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad']
print(classes[label])


5
Sad
